##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Ozel Katmanlar

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/tr/r1/tutorials/eager/custom_layers.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/tr/r1/tutorials/eager/custom_layers.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

Sinir aglarini olustururken yuksek seviye API olan 'tf.keras'i kullanmanizi tavsiye ederiz, ama bircok TensorFlow APIsi eager modunda rahatlikla kullanilabilir.


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

tf.enable_eager_execution()

## Katmanlar: kullanisli operasyonlarin bir araya geldigi setler

Makine ogrenmesi modelleri icin kod yazarken degiskenleri tek tek kontrol etmek ya da operasyonlari tek tek yonetmek yerine yuksek seviyede soyut dusunmeyi tercih ederiz.

Bircok makine ogrenmesi modeli basit katmanlarin ust uste binmesi ve birlestirilmesi olarak aciklanabilir. TensorFlow hem cok sayida genel katmanlar sunar hem de kendi uygulamaniza ozel katmanlari ya sifirdan ya da var olan katmanlara eklemeler yaparak kolayca olusturmanizi saglar.

[Keras](https://keras.io) API'si TensorFlow icindeki tf.keras paketinden bulunabilir ve Keras katmanlari kendi modellerinizi olustururken kullanabilirsiniz.


In [0]:
# tf.keras.layers paketinde katmanlar birer nesnedir. Katmanlari olusturmak icin,
# nesneyi olusturmaniz yeterlidir. Bircok katman cikti boyutlarinin ya da kanallarin sayisini 
# ilk bagimsiz degisken olarak alir.
layer = tf.keras.layers.Dense(100)
# Girdi boyutlarinin sayisi ise katman ilk kullanildiginda anlasilabilecegi icin cogunlukla gereksizdir,
# fakat isterseniz bunu ozellikle belirtebilirsiniz ki bu kompleks modellerde faydali olabilir
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))

Var olan katmanlarin tam listesi icin [buraya bakiniz](https://www.tensorflow.org/api_docs/python/tf/keras/layers). Bunlardan bazilari Dense: (tam-bagli katman),
Conv2D, LSTM, BatchNormalization, Dropout.

In [0]:
# Katmani kullanmak icin cagirmaniz yeterlidir.
layer(tf.zeros([10, 5]))

In [0]:
# Katmanlarin bircok kullanisli yontemleri vardir. Ornegin, butun katmandaki degiskenleri 
# `layer.variables` yontemi, egitebilir degiskenleri ise `layer.trainable_variables` 
# yontemi ile inceleyebilirsiniz. Burada ise tam-bagli bir katman agirlik ve sapma degiskenlerine sahiptir.
layer.variables

In [0]:
# Degiskenlere ayni zamanda 'nice accessors' ile erisebiliriz
layer.kernel, layer.bias

## Ozel katmanlari gerceklestirelim
tf.keras.Layer sinifini kullanip gelistirmek kendi katmanlarinizi olusturmanin en iyi yoludur:
  *  `__init__` , girdiye bagli olamayan ilklendirmeleri burda yapabilirsiniz
  * `build`, burada girdi tensorlarinin seklini bildiginiz icin geri kalan ilkelendirmeleri yapabilirsiniz
  * `call`, ileri hesaplamalari yapabildigimiz yer

Degiskenlerinizi olusturmak icin `build` asamasini beklemenize gerek yoktur, onlari `__init__` asamasinda da olusturabilirsiniz. Fakat, `build` asamasinda olusturulan degiskenler katmanlarin uzerinde calisacagi girdilerin seklini bilme avantajina sahiptir. Degiskenlerin `__init__` evresinde olusturulmasi ise bu sekillerin ozellikle belirtilmesini gerektirir.

In [0]:
class MyDenseLayer(tf.keras.layers.Layer):
  def __init__(self, num_outputs):
    super(MyDenseLayer, self).__init__()
    self.num_outputs = num_outputs

  def build(self, input_shape):
    self.kernel = self.add_variable("kernel",
                                    shape=[int(input_shape[-1]),
                                           self.num_outputs])

  def call(self, input):
    return tf.matmul(input, self.kernel)

layer = MyDenseLayer(10)
print(layer(tf.zeros([10, 5])))
print(layer.trainable_variables)

Kullanicilar standart katmanlarin davranisina asina oldugu icin mumkun oldugunca bunlari kullanmak kodunuzun okunabilirligini ve bakimini kolaylastiracaktir. tf.keras.layers ya da tf.contrib.layers icerisinde bulunmayan bir katman kullanmak istiyorsaniz [buraya](http://github.com/tensorflow/tensorflow/issues/new) basvurmanizi ya da daha iyisi bize bir 'pull request' gondermenizi tavsiye ederiz!

## Modeller: katmanlarin birlestirilmesi

Makine ogrenmesi modellerindeki ilginc katman benzeri seyler var olan katmanlarin birlestirilmesi ile olusturulur. Ornegin, 'resnet'teki her obek kivrimlarin, toplu normallestirmelerin ve kisa yollarin birlesmesi ile olusmustur.

tf.keras.Model sinif diger katmnalari iceren katman benzeri seylerin olusturulmasinda kullanilir. tf.keras.Model sinifindan kalit alarak bunu yapabilirsiniz.

In [0]:
class ResnetIdentityBlock(tf.keras.Model):
  def __init__(self, kernel_size, filters):
    super(ResnetIdentityBlock, self).__init__(name='')
    filters1, filters2, filters3 = filters

    self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
    self.bn2a = tf.keras.layers.BatchNormalization()

    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
    self.bn2b = tf.keras.layers.BatchNormalization()

    self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
    self.bn2c = tf.keras.layers.BatchNormalization()

  def call(self, input_tensor, training=False):
    x = self.conv2a(input_tensor)
    x = self.bn2a(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2b(x)
    x = self.bn2b(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2c(x)
    x = self.bn2c(x, training=training)

    x += input_tensor
    return tf.nn.relu(x)


block = ResnetIdentityBlock(1, [1, 2, 3])
print(block(tf.zeros([1, 2, 3, 3])))
print([x.name for x in block.trainable_variables])

Cogunlukla bircok katmandan olusan modeller sadece katmanlari belli bir sirayla cagirirlar. Bu islem tf.keras.Sequential kullanilarak cok az bir kod ile yapilabilir.

In [0]:
 my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1, 1)),
                               tf.keras.layers.BatchNormalization(),
                               tf.keras.layers.Conv2D(2, 1,
                                                      padding='same'),
                               tf.keras.layers.BatchNormalization(),
                               tf.keras.layers.Conv2D(3, (1, 1)),
                               tf.keras.layers.BatchNormalization()])
my_seq(tf.zeros([1, 2, 3, 3]))

# Sirada ne var

Simdi bir onceki kitapciga donerek dogrusal gerileme ornegini katmanlari ve modelleri kullanarak daha iyi yapilandirabilirsiniz.